# Before your start:
- Read the README.md file
- Comment as much as you can and use the resources (README.md file)
- Happy learning!

In [74]:
import pandas as pd
import numpy as np

import scipy.stats as st


# Challenge 1 - Exploring the Data

In this challenge, we will examine all salaries of employees of the City of Chicago. We will start by loading the dataset and examining its contents.

In [75]:
salaries = pd.read_csv("Current_Employee_Names__Salaries__and_Position_Titles.csv")

salaries

,Name,Job Titles,Department,Full or Part-Time,Salary or Hourly,Typical Hours,Annual Salary,Hourly Rate
0,"AARON, JEFFERY M",SERGEANT,POLICE,F,Salary,NaN,101442.0,NaN
1,"AARON, KARINA",POLICE OFFICER (ASSIGNED AS DETECTIVE),POLICE,F,Salary,NaN,94122.0,NaN
2,"AARON, KIMBERLEI R",CHIEF CONTRACT EXPEDITER,GENERAL SERVICES,F,Salary,NaN,101592.0,NaN
3,"ABAD JR, VICENTE M",CIVIL ENGINEER IV,WATER MGMNT,F,Salary,NaN,110064.0,NaN
4,"ABASCAL, REECE E",TRAFFIC CONTROL AIDE-HOURLY,OEMC,P,Hourly,20.0,NaN,19.86
...,...,...,...,...,...,...,...,...
33178,"ZYLINSKA, KATARZYNA",POLICE OFFICER,POLICE,F,Salary,NaN,72510.0,NaN
33179,"ZYMANTAS, LAURA C",POLICE OFFICER,POLICE,F,Salary,NaN,48078.0,NaN
33180,"ZYMANTAS, MARK E",POLICE OFFICER,POLICE,F,Salary,NaN,90024.0,NaN
33181,"ZYRKOWSKI, CARLO E",POLICE OFFICER,POLICE,F,Salary,NaN,93354.0,NaN


Examine the `salaries` dataset using the `head` function below.

In [76]:
salaries.head()

,Name,Job Titles,Department,Full or Part-Time,Salary or Hourly,Typical Hours,Annual Salary,Hourly Rate
0,"AARON, JEFFERY M",SERGEANT,POLICE,F,Salary,NaN,101442.0,NaN
1,"AARON, KARINA",POLICE OFFICER (ASSIGNED AS DETECTIVE),POLICE,F,Salary,NaN,94122.0,NaN
2,"AARON, KIMBERLEI R",CHIEF CONTRACT EXPEDITER,GENERAL SERVICES,F,Salary,NaN,101592.0,NaN
3,"ABAD JR, VICENTE M",CIVIL ENGINEER IV,WATER MGMNT,F,Salary,NaN,110064.0,NaN
4,"ABASCAL, REECE E",TRAFFIC CONTROL AIDE-HOURLY,OEMC,P,Hourly,20.0,NaN,19.86


We see from looking at the `head` function that there is quite a bit of missing data. Let's examine how much missing data is in each column. Produce this output in the cell below

In [77]:
null_cols = salaries.isnull().sum()
null_cols[null_cols >0]




Typical Hours    25161
Annual Salary     8022
Hourly Rate      25161
dtype: int64

Let's also look at the count of hourly vs. salaried employees. Write the code in the cell below

In [78]:
count = salaries.groupby(["Salary or Hourly"]).count()


count

,Name,Job Titles,Department,Full or Part-Time,Typical Hours,Annual Salary,Hourly Rate
Salary or Hourly,,,,,,,
Hourly,8022,8022,8022,8022,8022,0,8022
Salary,25161,25161,25161,25161,0,25161,0


What this information indicates is that the table contains information about two types of employees - salaried and hourly. Some columns apply only to one type of employee while other columns only apply to another kind. This is why there are so many missing values. Therefore, we will not do anything to handle the missing values.

There are different departments in the city. List all departments and the count of employees in each department.

In [79]:
count_dep= salaries.groupby(["Department"]).count()

count_dep["Name"]

Department
ADMIN HEARNG                39
ANIMAL CONTRL               81
AVIATION                  1629
BOARD OF ELECTION          107
BOARD OF ETHICS              8
BUDGET & MGMT               46
BUILDINGS                  269
BUSINESS AFFAIRS           171
CITY CLERK                  84
CITY COUNCIL               411
COMMUNITY DEVELOPMENT      207
COPA                       116
CULTURAL AFFAIRS            65
DISABILITIES                28
DoIT                        99
FAMILY & SUPPORT           615
FINANCE                    560
FIRE                      4641
GENERAL SERVICES           980
HEALTH                     488
HUMAN RELATIONS             16
HUMAN RESOURCES             79
INSPECTOR GEN               87
LAW                        407
LICENSE APPL COMM            1
MAYOR'S OFFICE              85
OEMC                      2102
POLICE                   13414
POLICE BOARD                 2
PROCUREMENT                 92
PUBLIC LIBRARY            1015
STREETS & SAN             21

# Challenge 2 - Hypothesis Tests

In this section of the lab, we will test whether the hourly wage of all hourly workers is significantly different from $30/hr. Import the correct one sample test function from scipy and perform the hypothesis test for a 95% two sided confidence interval.

In [103]:
# 1) hypothesis

# H0: hourly_wage = 30
# H1: hourly_wage != 30

# 2) significance:

alpha = 0.05

# 3) sample 

sample = salaries[salaries["Salary or Hourly"]== "Hourly"]["Hourly Rate"]

sample


4        19.86
6        46.10
7        35.60
10        2.65
18       17.68
         ...  
33164    46.10
33168    17.68
33169    35.60
33174    46.35
33175    48.85
Name: Hourly Rate, Length: 8022, dtype: float64

In [104]:
# 4) Compute statistics 

st.ttest_1samp(sample, 30)

#st.ttest_1samp(c3_sample, 17)

TtestResult(statistic=20.6198057854942, pvalue=4.3230240486229894e-92, df=8021)

In [82]:
# p_value e less than alpha, so i have to reject H0. hourly rate of the workers is 
# different from 30$

We are also curious about salaries in the police force. The chief of police in Chicago claimed in a press briefing that salaries this year are higher than last year's mean of $86000/year a year for all salaried employees. Test this one sided hypothesis using a 95% confidence interval.

Hint: A one tailed test has a p-value that is half of the two tailed p-value. If our hypothesis is greater than, then to reject, the test statistic must also be positive.

In [105]:
# H0: salaries >= 86
# H1: salaries < 86

alpha = 0.05

# using a sample

sample_salaries = salaries[salaries["Salary or Hourly"] == "Salary"]["Annual Salary"]


st.ttest_1samp(sample_salaries, 86, alternative = "less")

#st.ttest_1samp(c3_age_sample, 17, alternative = "greater")

TtestResult(statistic=653.6034847441706, pvalue=1.0, df=25160)

In [135]:
sample_salaries = salaries[salaries["Salary or Hourly"] == "Salary"][salaries["Department"]== "POLICE"]["Annual Salary"]


/var/folders/ms/vrwr3_md2xl156mcvfmyskx80000gn/T/ipykernel_77248/27391849.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  sample_salaries = salaries[salaries["Salary or Hourly"] == "Salary"][salaries["Department"]== "POLICE"]["Annual Salary"]


In [84]:
## p_value greater than significance level, so i can NOT reject H0, salaries seem to be 
# greater than 86000/year.

In [106]:
# using total workers from the dataframe that has a salary.

salaries1 = salaries[salaries["Salary or Hourly"] == "Salary"]["Annual Salary"]

st.ttest_1samp(salaries1, 86, alternative = "less")


TtestResult(statistic=653.6034847441706, pvalue=1.0, df=25160)

Using the `crosstab` function, find the department that has the most hourly workers. 

In [99]:
dep_hourly = pd.crosstab(salaries["Department"], salaries["Salary or Hourly"])

dep_max = dep_hourly.idxmax()

dep_max

Salary or Hourly
Hourly    STREETS & SAN
Salary           POLICE
dtype: object

In [100]:
# The departmente with the most hourly workers is Streets & San. 

In [101]:
salaries

,Name,Job Titles,Department,Full or Part-Time,Salary or Hourly,Typical Hours,Annual Salary,Hourly Rate
0,"AARON, JEFFERY M",SERGEANT,POLICE,F,Salary,NaN,101442.0,NaN
1,"AARON, KARINA",POLICE OFFICER (ASSIGNED AS DETECTIVE),POLICE,F,Salary,NaN,94122.0,NaN
2,"AARON, KIMBERLEI R",CHIEF CONTRACT EXPEDITER,GENERAL SERVICES,F,Salary,NaN,101592.0,NaN
3,"ABAD JR, VICENTE M",CIVIL ENGINEER IV,WATER MGMNT,F,Salary,NaN,110064.0,NaN
4,"ABASCAL, REECE E",TRAFFIC CONTROL AIDE-HOURLY,OEMC,P,Hourly,20.0,NaN,19.86
...,...,...,...,...,...,...,...,...
33178,"ZYLINSKA, KATARZYNA",POLICE OFFICER,POLICE,F,Salary,NaN,72510.0,NaN
33179,"ZYMANTAS, LAURA C",POLICE OFFICER,POLICE,F,Salary,NaN,48078.0,NaN
33180,"ZYMANTAS, MARK E",POLICE OFFICER,POLICE,F,Salary,NaN,90024.0,NaN
33181,"ZYRKOWSKI, CARLO E",POLICE OFFICER,POLICE,F,Salary,NaN,93354.0,NaN


The workers from the department with the most hourly workers have complained that their hourly wage is less than $35/hour. Using a one sample t-test, test this one-sided hypothesis at the 95% confidence level.

In [108]:
# H0 = hourly rate =< 35
# H1 = hourly rate > 35

alpha = 0.05

workers_hourly = salaries[salaries["Salary or Hourly"]== "Hourly"]["Hourly Rate"]


st.ttest_1samp(workers_hourly, len(workers_hourly), alternative = "greater")

#st.ttest_1samp(salaries1, 86, alternative = "less")


TtestResult(statistic=-59075.69614118866, pvalue=1.0, df=8021)

In [ ]:
## P value is greater than alpha, so i can NOT reject H0.

# Challenge 3: To practice - Constructing Confidence Intervals

While testing our hypothesis is a great way to gather empirical evidence for accepting or rejecting the hypothesis, another way to gather evidence is by creating a confidence interval. A confidence interval gives us information about the true mean of the population. So for a 95% confidence interval, we are 95% sure that the mean of the population is within the confidence interval. 
).

To read more about confidence intervals, click [here](https://en.wikipedia.org/wiki/Confidence_interval).


In the cell below, we will construct a 95% confidence interval for the mean hourly wage of all hourly workers. 

The confidence interval is computed in SciPy using the `t.interval` function. You can read more about this function [here](https://docs.scipy.org/doc/scipy-0.14.0/reference/generated/scipy.stats.t.html).

To compute the confidence interval of the hourly wage, use the 0.95 for the confidence level, number of rows - 1 for degrees of freedom, the mean of the sample for the location parameter and the standard error for the scale. The standard error can be computed using [this](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.sem.html) function in SciPy.

In [114]:

hourly_wage_mean = salaries[salaries["Salary or Hourly"]=="Hourly"]["Hourly Rate"].mean()

hourly_wage_std = salaries[salaries["Salary or Hourly"]=="Hourly"]["Hourly Rate"].std()

n = len(salaries[salaries["Salary or Hourly"]=="Hourly"]["Hourly Rate"])


st.norm.interval(0.95, loc=hourly_wage_mean, scale = hourly_wage_std/np.sqrt(n))



(32.52349834813791, 33.05361708442256)

Now construct the 95% confidence interval for all salaried employeed in the police in the cell below.

In [130]:

salary_police_mean = salaries[salaries["Salary or Hourly"] == "Salary"][salaries["Department"] == "POLICE"]["Annual Salary"].mean()

salary_police_std = salaries[salaries["Salary or Hourly"] == "Salary"][salaries["Department"] == "POLICE"]["Annual Salary"].std()

n = len(salaries[salaries["Salary or Hourly"] == "Salary"][salaries["Department"] == "POLICE"]["Annual Salary"])

st.norm.interval(0.95, loc=salary_police_mean, scale = salary_police_std/np.sqrt(n))


/var/folders/ms/vrwr3_md2xl156mcvfmyskx80000gn/T/ipykernel_77248/4047882935.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  salary_police_mean = salaries[salaries["Salary or Hourly"] == "Salary"][salaries["Department"] == "POLICE"]["Annual Salary"].mean()
/var/folders/ms/vrwr3_md2xl156mcvfmyskx80000gn/T/ipykernel_77248/4047882935.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  salary_police_std = salaries[salaries["Salary or Hourly"] == "Salary"][salaries["Department"] == "POLICE"]["Annual Salary"].std()
/var/folders/ms/vrwr3_md2xl156mcvfmyskx80000gn/T/ipykernel_77248/4047882935.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  n = len(salaries[salaries["Salary or Hourly"] == "Salary"][salaries["Department"] == "POLICE"]["Annual Salary"])


(86177.08425202279, 86795.74475424399)

In [126]:

## can i have two conditions on a boolean mask???? why is it giving this error???

salary_police_mean = salaries[salaries["Department"] == "POLICE"]["Annual Salary"].mean()

salary_police_mean

/var/folders/ms/vrwr3_md2xl156mcvfmyskx80000gn/T/ipykernel_77248/1646062075.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  salary_police_mean = salaries[salaries["Department"] == "POLICE"][salaries["Salary or Hourly"] == "Salary"]["Annual Salary"].mean()


86486.41450313339

# Bonus Challenge - Hypothesis Tests of Proportions

Another type of one sample test is a hypothesis test of proportions. In this test, we examine whether the proportion of a group in our sample is significantly different than a fraction. 

You can read more about one sample proportion tests [here](http://sphweb.bumc.bu.edu/otlt/MPH-Modules/BS/SAS/SAS6-CategoricalData/SAS6-CategoricalData2.html).

In the cell below, use the `proportions_ztest` function from `statsmodels` to perform a hypothesis test that will determine whether the number of hourly workers in the City of Chicago is significantly different from 25% at the 95% confidence level.

In [92]:
# Your code here:

